In [1]:
import json
import os
import mysql.connector

In [2]:
mydb=mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="Mysql123",
    database="Testdb2"
)
print(mydb)
crs=mydb.cursor()

In [3]:
def insertImage(name,id,text):
    
    insert_img='''
    INSERT INTO Images(img_id,img_name,text) VALUES (%s, %s, %s)
    '''
    data=(id,name,text)
    crs.execute(insert_img,data)

In [4]:
def insertPage(page_id,confidence,languages,coor,image_id):
    lang=languages.get('languageCode')
    
    coordinates=' '.join(str(coord) for coord in coor.values())

    data=(page_id,confidence,lang,coordinates,image_id)
    

    insert_page='''
    INSERT INTO Pages(page_id,page_confidence,page_languages,page_coor,img_id) VALUES (%s, %s, %s, %s, %s)
    '''
    crs.execute(insert_page,data)

In [5]:
def insertBlock(block_id,confidence,coor,page_id):
    coordinates=' '.join(str(coord) for coord in coor.values())

    data=(block_id,confidence,coordinates,page_id)
    
    insert_block='''
    INSERT INTO Blocks(block_id,block_confidence,block_coor,page_id) VALUES (%s, %s, %s, %s)
    '''
    crs.execute(insert_block,data)

In [6]:
def insertPara(para_id,confidence,coor,block_id):
    coordinates=' '.join(str(coord) for coord in coor.values())

    data=(para_id,confidence,coordinates,block_id)
    
    insert_para='''
    INSERT INTO Paragraphs(para_id,para_confidence,para_coor,block_id) VALUES (%s, %s, %s, %s)
    '''
    crs.execute(insert_para,data) 

In [7]:
def insertWord(word_id,word,confidence,languages,coor,para_id):
    lang=languages.get('languageCode')
    
    coordinates=' '.join(str(coord) for coord in coor.values())

    data=(word_id,word,confidence,lang,coordinates,para_id)
    

    insert_word='''
    INSERT INTO Words(word_id,word_text,word_confidence,word_languages,word_coor,para_id) VALUES (%s, %s, %s, %s, %s, %s)
    '''
    crs.execute(insert_word,data) 

In [8]:
def insertSymbol(symbol_id,symbol,confidence,coor,word_id):
    
    coordinates=' '.join(str(coord) for coord in coor.values())

    data=(symbol_id,symbol,confidence,coordinates,word_id)
    

    insert_symbol='''
    INSERT INTO Symbols(symbol_id,symbol_text,symbol_confidence,symbol_coor,word_id) VALUES (%s, %s, %s, %s, %s)
    '''
    crs.execute(insert_symbol,data)

In [9]:
def sqlInjection(data,name):
    global img_id
    global page_id
    global block_id
    global para_id
    global word_id
    global symbol_id
    
    text=data['fullTextAnnotation']['text']
    insertImage(name,img_id,text)

    for page in data['fullTextAnnotation']['pages']:
        
        page_id+=1
        page_width=page.get('width')
        page_height=page.get('height')

        page_confidence=page['confidence']
        page_coor={0:[0,0],1:[page_width,0],2:[page_width,page_height],3:[0,page_height]}
        
        page_languages={}

        if(page.get('property')):
            for detectedLanguage in page['property']['detectedLanguages']:
                page_languages=detectedLanguage

        insertPage(page_id,page_confidence,page_languages,page_coor,img_id)
        
        for block in page['blocks']:
            block_id+=1
            
            block_confidence=block['confidence']

            block_coor={}
            for index,vertice in enumerate(block['boundingBox']['vertices']):
                block_coor[index]=[vertice.get('x'),vertice.get('y')]

            insertBlock(block_id,block_confidence,block_coor,page_id)

            for paragraph in block['paragraphs']:
                para_id+=1

                para_confidence=paragraph['confidence']

                para_coor={}
                for index,vertice in enumerate(paragraph['boundingBox']['vertices']):
                    para_coor[index]=[vertice.get('x'),vertice.get('y')]

                insertPara(para_id,para_confidence,para_coor,block_id)
                
                for word in paragraph['words']:
                    word_id+=1
                    
                    word_languages={}

                    if(word.get('property')):
                        for detectedLanguage in word['property']['detectedLanguages']:
                            word_languages=detectedLanguage

                    word_confidence=word['confidence']

                    word_coor={}
                    for index,vertice in enumerate(word['boundingBox']['vertices']):
                        word_coor[index]=[vertice.get('x'),vertice.get('y')]
                    
                    word_text = ''.join([symbol['text'] for symbol in word['symbols']])
                    
                    insertWord(word_id,word_text,word_confidence,word_languages,word_coor,para_id)

                    for symbol in word['symbols']:
                        symbol_id+=1

                        symbol_confidence=symbol['confidence']

                        symbol_coor={}
                        for index,vertice in enumerate(symbol['boundingBox']['vertices']):
                            symbol_coor[index]=[vertice.get('x'),vertice.get('y')]
                        
                        symbol_text =symbol['text']

                        insertSymbol(symbol_id,symbol_text,symbol_confidence,symbol_coor,word_id)

In [10]:
directory='/Users/joshvirnarula/snapwrite/misc/sentence_testing_json/'

img_id=0
page_id=0
block_id=0
para_id=0
word_id=0
symbol_id=0
for img_id,file in enumerate(os.listdir(directory),1):
    file_name = os.path.join(directory, file)
    with open(file_name,'r') as f:
        try:
            data=json.load(f)
            img_name='Image'+str(img_id)
            sqlInjection(data,img_name)
            #print("processed" , img_id , "images..")
        except:
            print("utf-8 could not correctly encode json file",img_id)

utf-8 could not correctly encode json file 11
utf-8 could not correctly encode json file 65


In [11]:
mydb.commit()